<a href="https://colab.research.google.com/github/tiara070403/seminar/blob/main/seminar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn
!pip install Sastrawi
!pip install transformers
# Import Library
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from wordcloud import WordCloud
import nltk
nltk.download('punkt')
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from transformers import pipeline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.3 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df = pd.read_csv('/content/crawling.csv', index_col=0)

In [ ]:
df = df[['created_at', 'full_text']]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5349 entries, 1858070162362204643 to 1230765438897815552
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   created_at  5349 non-null   object
 1   full_text   5349 non-null   object
dtypes: object(2)
memory usage: 125.4+ KB


In [ ]:
df['full_text'] = df['full_text'].replace('false', pd.NA)
df = df.dropna(subset=['full_text'])

In [ ]:
# Menghapus Duplikat
initial_duplicates = df.duplicated().sum()
df = df.drop_duplicates(subset=['full_text'])
print(f"Jumlah duplikat awal: {initial_duplicates}")
print(f"Jumlah duplikat setelah drop: {df.duplicated().sum()}")

Jumlah duplikat awal: 4
Jumlah duplikat setelah drop: 0


In [ ]:
# Menghapus baris dengan nilai null (setelah dropna subset)
print(f"Jumlah nilai null setelah dropna: \n{df.isnull().sum()}")

Jumlah nilai null setelah dropna: 
created_at    0
full_text     0
dtype: int64


In [ ]:
import pandas as pd

# 1) Buka file
df = pd.read_csv("crawling.csv")

# 2) Cek ukuran data & nama kolom
print("shape:", df.shape)
print("columns:", list(df.columns))

# 3) Lihat 5 baris teratas
df.head()

shape: (5349, 15)
columns: ['conversation_id_str', 'created_at', 'favorite_count', 'full_text', 'id_str', 'image_url', 'in_reply_to_screen_name', 'lang', 'location', 'quote_count', 'reply_count', 'retweet_count', 'tweet_url', 'user_id_str', 'username']


,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1858070162362204643,Sun Nov 17 08:50:22 +0000 2024,2,lucu banget mahasiswa pencinta alam ini,1858070162362204643,NaN,NaN,in,NaN,0,1,0,https://x.com/undefined/status/185807016236220...,1716269490936360960,NaN
1,1846529554657239219,Wed Oct 16 12:32:07 +0000 2024,0,Bejat Mahasiswa Pencinta Alam di Jambi 'Genjot...,1846529554657239219,NaN,NaN,in,NaN,0,0,0,https://x.com/undefined/status/184652955465723...,2544978158,NaN
2,1467691301155979267,Wed Oct 09 13:58:44 +0000 2024,0,@Leonita_Lestari Ada Perkoempoelan Pentjinta A...,1844014639350452258,NaN,Leonita_Lestari,in,NaN,0,0,0,https://x.com/undefined/status/184401463935045...,1810821796767375360,NaN
3,1836014724523458593,Tue Sep 17 12:09:56 +0000 2024,0,[PRESS RELEASE SOWAN UKM KE UKM UNIT PANDU LIN...,1836014724523458593,https://pbs.twimg.com/media/GXrVp2Wa0AAv3zG.jpg,NaN,in,NaN,0,1,0,https://x.com/undefined/status/183601472452345...,297204645,NaN
4,1835164951125917967,Sun Sep 15 03:53:14 +0000 2024,0,[PRESS RELEASE SOWAN UKM KE UKM UNIT PANDU LIN...,1835164951125917967,https://pbs.twimg.com/media/GXfQzAtacAAlymL.jpg,NaN,in,NaN,0,1,0,https://x.com/undefined/status/183516495112591...,297204645,NaN


In [ ]:
# read 'full_text' column saja
df["full_text"].head()

,full_text
0,lucu banget mahasiswa pencinta alam ini
1,Bejat Mahasiswa Pencinta Alam di Jambi 'Genjot...
2,@Leonita_Lestari Ada Perkoempoelan Pentjinta A...
3,[PRESS RELEASE SOWAN UKM KE UKM UNIT PANDU LIN...
4,[PRESS RELEASE SOWAN UKM KE UKM UNIT PANDU LIN...


In [ ]:
!pip install tqdm
from tqdm.notebook import tqdm

tqdm.pandas()

In [ ]:
import re
import pandas as pd

# load data
df = pd.read_csv("crawling.csv")

# load slang.csv (format: slang,formal)
slang_dict = pd.read_csv("slang.csv")
slang_map = dict(zip(slang_dict['slang'], slang_dict['formal']))

def preprocess_sentiment_no_stem(text):
      text = str(text).lower()                        # case folding
      text = re.sub(r'@\w+', ' ', text)               # hapus mention
      text = re.sub(r'http\S+|www.\S+', ' ', text)    # hapus URL
      text = re.sub(r'[^a-z\s]', ' ', text)           # hapus selain huruf
      text = re.sub(r'\s+', ' ', text).strip()        # rapikan spasi

      tokens = text.split()
      tokens = [slang_map.get(w, w) for w in tokens]  # normalisasi slang

      return " ".join(tokens)

# terapkan
df["clean_text"] = df["full_text"].apply(preprocess_sentiment_no_stem)

# simpan
# cek hasil
df[["full_text", "clean_text"]].head(10)
df[["full_text", "clean_text"]].to_csv("/content/clean_crawling.csv", index=False)

print("✅ Selesai! Hasil disimpan di /content/clean_crawling.csv")

✅ Selesai! Hasil disimpan di /content/clean_crawling.csv


In [ ]:
df = pd.read_csv("/content/clean_crawling.csv")

In [ ]:
!pip install Sastrawi tqdm
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tqdm.notebook import tqdm

# buat stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

tqdm.pandas()

def stem_only(text):
      tokens = str(text).split()
      tokens = [stemmer.stem(w) for w in tokens]
      return " ".join(tokens)

# buat kolom baru hasil stemming
df["stemmed_text"] = df["clean_text"].progress_apply(stem_only)

# simpan hasil
df.to_csv("clean_crawling_stemmed.csv", index=False)

print("✅ Stemming selesai! Hasil ada di clean_crawling_stemmed.csv")


  0%|          | 0/5349 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
df = pd.read_csv("/content/clean_crawling_stemmed.csv")

In [ ]:
lexicon = pd.read_csv("/content/full_lexicon.csv")
def label_text(text):
    labels = []
    words = text.split()
    for word in words:
        if word in lexicon['word'].values:
            label = lexicon.loc[lexicon['word'] == word, 'weight'].values[0]
            labels.append(label)
    return labels
df['labels'] = df['clean_text'].apply(label_text)